In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
import pickle as pkl

In [4]:
%run utils.ipynb

In [5]:
df = pd.read_csv('data/ETHBTC_1m_1519496760000_1549568220000.csv')
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [6]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE
0  1519496760000  0.085541  0.085570  0.085407  0.085473
1  1519496820000  0.085473  0.085570  0.085448  0.085449
2  1519496880000  0.085448  0.085569  0.085400  0.085538
3  1519496940000  0.085538  0.085546  0.085402  0.085540
4  1519497000000  0.085539  0.085552  0.085464  0.085464

In [7]:
data_validation(data)

In [8]:
# Get Target
%run targets.ipynb
target = get_target_1()
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [9]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape

(499999, 6)

In [10]:
data_validation(data)

In [11]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299999, 6)

(100000, 6)

(100000, 6)

In [12]:
# Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = scaler.fit(TR_data.iloc[:, 1:-1].values)

TR_data.iloc[:, 1:-1] = scaler.transform(TR_data.iloc[:, 1:-1])
CV_data.iloc[:, 1:-1] = scaler.transform(CV_data.iloc[:, 1:-1])
TT_data.iloc[:, 1:-1] = scaler.transform(TT_data.iloc[:, 1:-1])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=120, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=120, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=120, tt_win=0, point_target=True)

In [14]:
TR_X.shape
TR_y.shape

(299880, 481)

(299880,)

In [15]:
np.unique(TR_y)

array([0., 1.])

In [ ]:
## Not Working
# from sklearn.linear_model import ARDRegression
# 
# ardmodel = ARDRegression(
#     n_iter=300, 
#     tol=0.001, 
#     alpha_1=1e-06, 
#     alpha_2=1e-06, 
#     lambda_1=1e-06, 
#     lambda_2=1e-06, 
#     compute_score=False, 
#     threshold_lambda=10000.0, 
#     fit_intercept=True, 
#     normalize=False, 
#     copy_X=True, 
#     verbose=True)
# ardmodel = ardmodel.fit(TR_X, TR_y)

In [20]:
from sklearn.svm import LinearSVC

modelsvc = LinearSVC(
    penalty='l2', 
    loss='squared_hinge', 
    dual=True, 
    tol=0.0001, 
    C=1.0, 
    multi_class='ovr', 
    fit_intercept=True, 
    intercept_scaling=1, 
    class_weight=None, 
    verbose=1, 
    random_state=0, 
    max_iter=1000
)
modelsvc = modelsvc.fit(TR_X[:, 1:], TR_y.astype(int))

[LibLinear]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [24]:
modelsvc.predict(CV_X[:, 1:])

array([1, 1, 0, ..., 1, 1, 1])

In [21]:
%run targets.ipynb
models = {
    'modelsvc': modelsvc
}

In [22]:
tr_res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
for model_name, model in models.iteritems():
    if model_name in ['l2lrmodel', 'nbmodel']:
        TR_y_pred = model.predict_proba(TR_X[:, 1:])[:, 1]
    else:
        TR_y_pred = model.predict(TR_X[:, 1:])
    metric_scores = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    tr_res = tr_res.append(pd.Series(metric_scores, name=model_name))

tr_res

acc       auc        f1   log_loss
modelsvc  0.52525  0.511764  0.265372  16.397345

In [23]:
cv_res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
for model_name, model in models.iteritems():
    if model_name in ['l2lrmodel', 'nbmodel']:
        CV_y_pred = model.predict_proba(CV_X[:, 1:])[:, 1]
    else:
        CV_y_pred = model.predict(CV_X[:, 1:])
    metric_scores = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    cv_res = cv_res.append(pd.Series(metric_scores, name=model_name))

cv_res

acc       auc        f1   log_loss
modelsvc  0.491475  0.495766  0.523116  17.564085

In [ ]:
tt_res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
for model_name, model in models.iteritems():
    if model_name in ['l2lrmodel', 'nbmodel']:
        TT_y_pred = model.predict_proba(TT_X[:, 1:])[:, 1]
    else:
        TT_y_pred = model.predict(TT_X[:, 1:])
    metric_scores = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    tt_res = tt_res.append(pd.Series(metric_scores, name=model_name))

tt_res